In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#import mutagen
import os
import random
import wave
import numpy as np
from scipy.io.wavfile import read as read_wav
from scipy.io.wavfile import write as write_wav
import librosa
import librosa.display
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import matplotlib.pyplot as plt

import utils
import classifier
import multi_classifier as mclass
import irmasTrainUtils as trainUtils
import irmasTestUtils as testUtils

In [4]:
#trainUtils.parse_irmas_trainset("IRMAS-TrainingData", "Preprocessed_Trainset")

In [5]:
#df_train = trainUtils.load_train_dataset("Preprocessed_Trainset/Train")
#df_valid = trainUtils.load_train_dataset("Preprocessed_Trainset/Validation")

In [6]:
#df_train.to_pickle("Preprocessed_Trainset/pickled_train.pkl")
#df_valid.to_pickle("Preprocessed_Trainset/pickled_validation.pkl")

In [3]:
df_train = utils.load_pickled_dataset("Preprocessed_Trainset/train.pkl")
df_valid = utils.load_pickled_dataset("Preprocessed_Trainset/validation.pkl")
df_train = df_train.sample(frac=1)
df_valid = df_valid.sample(frac=1)

In [4]:
model = classifier.make_model()

In [9]:
#classifier.train_model(model, df_train, batch_size=32, epochs=30)

In [5]:
model.load_weights("Saved_Weights/weights")

In [6]:
print(classifier.get_accuracy(model, df_train))

[3]
0.7036668325866932


In [7]:
print(classifier.get_accuracy(model, df_valid))

[3]
0.6047197640117994


In [ ]:
print(model.history)

In [8]:
multi_model = mclass.make_model()
mclass.load_weights(multi_model, model)

In [9]:
mclass.train_model(multi_model, df_train, batch_size=32, epochs=30)

Epoch 1/30
377/377 [==============================] - 86s 213ms/step - loss: 2.1861
Epoch 2/30
377/377 [==============================] - 98s 260ms/step - loss: 1.4853
Epoch 3/30
377/377 [==============================] - 100s 265ms/step - loss: 1.3472
Epoch 4/30
377/377 [==============================] - 90s 240ms/step - loss: 1.2451
Epoch 5/30
377/377 [==============================] - 92s 243ms/step - loss: 1.2276
Epoch 6/30
377/377 [==============================] - 78s 207ms/step - loss: 1.1741
Epoch 7/30
377/377 [==============================] - 81s 215ms/step - loss: 1.1497
Epoch 8/30
377/377 [==============================] - 83s 221ms/step - loss: 1.1144
Epoch 9/30
377/377 [==============================] - 77s 204ms/step - loss: 1.1165
Epoch 10/30
377/377 [==============================] - 78s 207ms/step - loss: 1.0949
Epoch 11/30
377/377 [==============================] - 83s 219ms/step - loss: 1.0679
Epoch 12/30
377/377 [==============================] - 80s 212ms/step - l

In [13]:
print(mclass.get_accuracy(multi_model, df_train))

0.0909090909090909


In [14]:
print(mclass.get_accuracy(multi_model, df_valid))

0.09090909090909094


In [12]:

x, y = np.stack(df_train.data), np.stack(df_train.label)
x = x.reshape((-1,128,259,2))
print(multi_model.predict(x)[0])

[0.86317337 0.36226666 0.0948779  0.14307916 0.30608308 0.06918737
 0.21820486 0.82087505 0.53484875 0.88276625 0.01191196]


In [ ]:



smaller = mclass.make_model()
for layer in test.layers:
    for i in range(len(smaller)):
        for v in range(len(smaller[i].layers)):
            if smaller[i].layers[v].name == layer.name:
                smaller[i].layers[v].set_weights(layer.get_weights())

In [ ]:
print(smaller[0].layers)

In [ ]:
x, ys = mclass.process_dataframe(df_train)

In [ ]:
mclass.train_model(smaller, x, ys, batch_size=32, epochs=50)

In [ ]:
model = tf.keras.Sequential()

model.add(tf.keras.layers.Conv2D(filters=8, kernel_size=4, strides=(2,2), padding='same', input_shape=(128,259,1), name="conv_1"))
model.add(tf.keras.layers.BatchNormalization(axis=1, name="bn_1"))
model.add(tf.keras.layers.ReLU())
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2), name="mpool_1"))

model.add(tf.keras.layers.Conv2D(filters=16, kernel_size=3, strides=(1,1), padding='valid', name="conv_2"))
model.add(tf.keras.layers.BatchNormalization(axis=1, name="bn_2"))
model.add(tf.keras.layers.ReLU())
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2), name="mpool_2"))

model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=2, strides=(1,1), padding='valid', name="conv_3"))
model.add(tf.keras.layers.BatchNormalization(axis=1, name="bn_3"))
model.add(tf.keras.layers.ReLU())
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2), name="mpool_3"))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(500, activation='relu', name="fc_4"))
model.add(tf.keras.layers.Dense(11, activation=None, name="fc_5"))
model.add(tf.keras.layers.Softmax())

model.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy())

train_set = df.tail(-1000)
test_set = df.head(1000)

x, y = np.stack(train_set.data), np.stack(train_set.label)
x = x.reshape((-1,128,259,1))
print(x.shape)
print(x.shape, y.shape)

model.fit(x, y, batch_size=32, epochs=30)

'''x, y = np.stack(sample['data']), np.stack(sample['label'])
print(x.shape, y.shape)
print(type(x), type(y))
x, y = tf.convert_to_tensor(x, dtype=tf.float32), tf.convert_to_tensor(y, dtype=tf.float32)
model.predict(x)'''

In [ ]:
x, y = np.stack(train_set.data), np.stack(train_set.label)
x = x.reshape((-1,128,259,1))

preds = np.argmax(model.predict(x), axis=1).reshape(x.shape[0], 1)
true = np.argmax(y, axis=1)

print("Train accuracy:", np.sum((preds == true)*np.ones(preds.shape))/x.shape[0])

In [ ]:
x, y = np.stack(test_set.data), np.stack(test_set.label)
x = x.reshape((-1,128,259,1))

preds = np.argmax(model.predict(x), axis=1).reshape(x.shape[0], 1)
true = np.argmax(y, axis=1)

print("Test accuracy:", np.sum((preds == true)*np.ones(preds.shape))/x.shape[0])

In [ ]:
#15 epochs - train 88.0 test 51.5


#30 epochs - train accuracy 96.7%, test accuracy 36.8%
#20 epochs - train accuracy 98.8%, test accuracy 47.8%
#15 epochs - train accuracy 97.5%, test accuracy 36.4%
#10 epochs - train accuracy 95.7%, test accuracy 44.4%

arr = np.array([[5, 2], [3, 4], [6, 8]])
print (arr)
print (np.linalg.norm(arr))
print (arr / np.linalg.norm(arr))

In [ ]:
utils.combine_dataset(df, 5, 3)